# Demos: Lecture 6

Note: the code included here is similar (but not identical) to the code I showed in CircInspect during class. 

## Demo 1: Superdense coding

<img src="fig/superdense.png" width="600px"> 

In [1]:
import pennylane as qml

dev = qml.device("default.qubit", wires=2, shots=1)

def create_entangled_state(wires):
    qml.Hadamard(wires=wires[0])
    qml.CNOT(wires=wires)

def bell_basis_rotation(wires):
    qml.adjoint(create_entangled_state)(wires)

def alice_chooses_bits(wire):
    b1, b2 = 0, 1
    if b1 == 1:
        qml.PauliZ(wires=wire)
    if b2 == 1:
        qml.PauliX(wires=wire)

@qml.qnode(dev)
def superdense_coding():
    create_entangled_state([0, 1])
    alice_chooses_bits(0)
    bell_basis_rotation([0, 1])
    return qml.sample()

# Execute a QNode to render a circuit in the righthand pane
superdense_coding()


tensor([0, 1], requires_grad=True)

## Demo 2: Teleportation

 <img src="fig/teleportation.png" width="800px">

In [3]:
dev = qml.device("default.qubit", wires=3)

def prepare_state(wire):
    qml.Hadamard(wires=wire)
    qml.RZ(0.2, wires=wire)
    qml.RY(0.8, wires=wire)

@qml.qnode(dev)
def teleportation():
    prepare_state(wire=0)
    create_entangled_state(wires=[1, 2])
    bell_basis_rotation(wires=[0, 1])

    m1 = qml.measure(wires=0)
    m2 = qml.measure(wires=1)

    qml.cond(m1 == 1, qml.PauliZ)(wires=2)
    qml.cond(m2 == 1, qml.PauliX)(wires=2)

    return qml.probs(wires=2)

# Execute a QNode to render a circuit in the righthand pane
teleportation()


tensor([0.14847164, 0.85152836], requires_grad=True)